# Explore here

It's recommended to use this notebook for exploration purposes.

In [2]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

url = "https://en.wikipedia.org/wiki/List_of_Spotify_streaming_records"
response = requests.get(url)

if response.status_code != 200:
    raise Exception(f"Error accessing page: {response.status_code}")
print ("status:", response.status_code)

status: 200


In [3]:
import pandas as pd

table = pd.read_html(url)
print(f"{len(table)} tables were found") 


27 tables were found
